<a href="https://colab.research.google.com/github/Sazul19/soil_analysis_automation_system/blob/main/enhanced_model_prototype_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0  # Import pre-trained EfficientNetB0 model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout  # Import necessary layers
from tensorflow.keras.models import Model  # Import Model API
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Import ImageDataGenerator for data augmentation
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint   # Import callbacks for training

# Load Pre-Trained EfficientNetB0 Model
# Function to Build the CNN Model
def build_model(num_classes):
    # Load EfficientNetB0 pre-trained model without top layers
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False    # Freeze the pre-trained model layers to retain learned features


    # Add Custom Layers for Color Classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Global average pooling layer to reduce dimensions
    x = Dropout(0.5)(x)  # Dropout to reduce overfitting
    x = Dense(128, activation='relu')(x)  # Fully connected layer with ReLU activation
    x = Dropout(0.3)(x)  # Another dropout layer
    predictions = Dense(num_classes, activation='softmax')(x)    # Output layer for classification
    model = Model(inputs=base_model.input, outputs=predictions) # Create the final model
    return model


# Prepare the Data
def prepare_data(train_dir, val_dir, batch_size=32, img_size=(224, 224)):
    datagen = ImageDataGenerator(
        rescale=1.0 / 255.0,   # Normalize pixel values between 0 and 1
        rotation_range=30,   # Random rotation for data augmentation
        width_shift_range=0.2,   # Horizontal shift
        height_shift_range=0.2,  # Vertical shift
        shear_range=0.2,# Shearing transformation
        zoom_range=0.2,  # Zoom transformation
        horizontal_flip=True, # Flip images horizontally
        validation_split=0.2 # Split 20% of data for validation
    )

    # Load training images
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',    # Multi-class classification
        subset='training'
    )

    # Load validation images
    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator


# Training Configuration
# Function to Train the Model
def train_model(model, train_generator, val_generator, epochs=20, batch_size=32, learning_rate=0.001):
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate), # Adam optimizer with learning rate 0.001
        loss='categorical_crossentropy',  # Loss function for multi-class classification
        metrics=['accuracy'] # Track accuracy
    )

    # Callbacks
    # Define Early Stopping and Model Checkpoint Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)   # Stop training early if no improvement
    model_checkpoint = ModelCheckpoint('efficientnet_color_model.h5', monitor='val_loss', save_best_only=True) # Save the best model

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_steps=val_generator.samples // batch_size,
        callbacks=[early_stopping, model_checkpoint]  # Use callbacks to improve training
    )

    return history


# Main Execution Block
if __name__ == "__main__":
    # Define paths to training and validation datasets
    train_dir = 'path/to/train/data'  # Replace with your train data path
    val_dir = 'path/to/validation/data'  # Replace with your validation data path

    # Prepare data
    train_generator, val_generator = prepare_data(train_dir, val_dir)

    # Build model
    num_classes = len(train_generator.class_indices)  # Dynamically calculate number of color categories
    model = build_model(num_classes)

    # Train the model
    history = train_model(model, train_generator, val_generator)

    # Unfreeze the base model for fine-tuning
    # Compile model again with a lower learning rate for fine-tuning
    model.layers[0].trainable = True
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model with a smaller learning rate
    history_fine = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,  # Fine-tune for 10 more epochs
        steps_per_epoch=train_generator.samples // 32,
        validation_steps=val_generator.samples // 32,
        callbacks=[early_stopping, model_checkpoint]
    )


ModuleNotFoundError: No module named 'tensorflow'